# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 7 2023 11:40AM,256825,20,ATR-I000096,"HVL, LLC dba Atrium Innovations",Released
1,Feb 7 2023 11:39AM,256824,19,9128126,"Emersa Waterbox, LLC",Released
2,Feb 7 2023 11:23AM,256823,19,MSP218262,"Methapharm, Inc.",Released
3,Feb 7 2023 11:23AM,256823,19,MSP218272,"Methapharm, Inc.",Released
4,Feb 7 2023 11:23AM,256823,19,MSP218279,"Methapharm, Inc.",Released
5,Feb 7 2023 11:07AM,256822,20,9128104,"Exact-Rx, Inc.",Released
6,Feb 7 2023 11:07AM,256822,20,9128105,"Exact-Rx, Inc.",Released
7,Feb 7 2023 11:07AM,256822,20,9128106,"Exact-Rx, Inc.",Released
8,Feb 7 2023 11:07AM,256822,20,9128107,"Exact-Rx, Inc.",Released
9,Feb 7 2023 11:07AM,256822,20,9128108,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,256821,Released,4
26,256822,Released,10
27,256823,Released,3
28,256824,Released,1
29,256825,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256821,NaN,NaN,4.0
256822,NaN,NaN,10.0
256823,NaN,NaN,3.0
256824,NaN,NaN,1.0
256825,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256777,0.0,0.0,1.0
256781,0.0,0.0,1.0
256782,0.0,84.0,21.0
256788,0.0,0.0,1.0
256791,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256777,0,0,1
256781,0,0,1
256782,0,84,21
256788,0,0,1
256791,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256777,0,0,1
1,256781,0,0,1
2,256782,0,84,21
3,256788,0,0,1
4,256791,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256777,,,1
1,256781,,,1
2,256782,,84,21
3,256788,,,1
4,256791,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 7 2023 11:40AM,256825,20,"HVL, LLC dba Atrium Innovations"
1,Feb 7 2023 11:39AM,256824,19,"Emersa Waterbox, LLC"
2,Feb 7 2023 11:23AM,256823,19,"Methapharm, Inc."
5,Feb 7 2023 11:07AM,256822,20,"Exact-Rx, Inc."
15,Feb 7 2023 11:06AM,256821,10,MedStone Pharma LLC
19,Feb 7 2023 10:55AM,256820,10,"ClearSpec, LLC"
23,Feb 7 2023 10:45AM,256819,10,Bio-PRF
32,Feb 7 2023 10:37AM,256817,10,ISDIN Corporation
36,Feb 7 2023 10:28AM,256816,10,Methapharm-G
39,Feb 7 2023 10:16AM,256814,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 7 2023 11:40AM,256825,20,"HVL, LLC dba Atrium Innovations",,,1
1,Feb 7 2023 11:39AM,256824,19,"Emersa Waterbox, LLC",,,1
2,Feb 7 2023 11:23AM,256823,19,"Methapharm, Inc.",,,3
3,Feb 7 2023 11:07AM,256822,20,"Exact-Rx, Inc.",,,10
4,Feb 7 2023 11:06AM,256821,10,MedStone Pharma LLC,,,4
5,Feb 7 2023 10:55AM,256820,10,"ClearSpec, LLC",,1,3
6,Feb 7 2023 10:45AM,256819,10,Bio-PRF,,,9
7,Feb 7 2023 10:37AM,256817,10,ISDIN Corporation,,,4
8,Feb 7 2023 10:28AM,256816,10,Methapharm-G,,,3
9,Feb 7 2023 10:16AM,256814,10,Eywa Pharma Inc.,,,17


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 11:40AM,256825,20,"HVL, LLC dba Atrium Innovations",1,,
1,Feb 7 2023 11:39AM,256824,19,"Emersa Waterbox, LLC",1,,
2,Feb 7 2023 11:23AM,256823,19,"Methapharm, Inc.",3,,
3,Feb 7 2023 11:07AM,256822,20,"Exact-Rx, Inc.",10,,
4,Feb 7 2023 11:06AM,256821,10,MedStone Pharma LLC,4,,
5,Feb 7 2023 10:55AM,256820,10,"ClearSpec, LLC",3,1,
6,Feb 7 2023 10:45AM,256819,10,Bio-PRF,9,,
7,Feb 7 2023 10:37AM,256817,10,ISDIN Corporation,4,,
8,Feb 7 2023 10:28AM,256816,10,Methapharm-G,3,,
9,Feb 7 2023 10:16AM,256814,10,Eywa Pharma Inc.,17,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 11:40AM,256825,20,"HVL, LLC dba Atrium Innovations",1,,
1,Feb 7 2023 11:39AM,256824,19,"Emersa Waterbox, LLC",1,,
2,Feb 7 2023 11:23AM,256823,19,"Methapharm, Inc.",3,,
3,Feb 7 2023 11:07AM,256822,20,"Exact-Rx, Inc.",10,,
4,Feb 7 2023 11:06AM,256821,10,MedStone Pharma LLC,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 11:40AM,256825,20,"HVL, LLC dba Atrium Innovations",1.0,NaN,NaN
1,Feb 7 2023 11:39AM,256824,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
2,Feb 7 2023 11:23AM,256823,19,"Methapharm, Inc.",3.0,NaN,NaN
3,Feb 7 2023 11:07AM,256822,20,"Exact-Rx, Inc.",10.0,NaN,NaN
4,Feb 7 2023 11:06AM,256821,10,MedStone Pharma LLC,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 11:40AM,256825,20,"HVL, LLC dba Atrium Innovations",1.0,0.0,0.0
1,Feb 7 2023 11:39AM,256824,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
2,Feb 7 2023 11:23AM,256823,19,"Methapharm, Inc.",3.0,0.0,0.0
3,Feb 7 2023 11:07AM,256822,20,"Exact-Rx, Inc.",10.0,0.0,0.0
4,Feb 7 2023 11:06AM,256821,10,MedStone Pharma LLC,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3595250,152.0,34.0,2.0
15,256782,21.0,84.0,0.0
16,256805,1.0,0.0,0.0
19,770438,4.0,1.0,0.0
20,770435,12.0,0.0,0.0
21,770427,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3595250,152.0,34.0,2.0
1,15,256782,21.0,84.0,0.0
2,16,256805,1.0,0.0,0.0
3,19,770438,4.0,1.0,0.0
4,20,770435,12.0,0.0,0.0
5,21,770427,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,152.0,34.0,2.0
1,15,21.0,84.0,0.0
2,16,1.0,0.0,0.0
3,19,4.0,1.0,0.0
4,20,12.0,0.0,0.0
5,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,152.0
1,15,Released,21.0
2,16,Released,1.0
3,19,Released,4.0
4,20,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0
Executing,34.0,84.0,0.0,1.0,0.0,0.0
Released,152.0,21.0,1.0,4.0,12.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,34.0,84.0,0.0,1.0,0.0,0.0
2,Released,152.0,21.0,1.0,4.0,12.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,34.0,84.0,0.0,1.0,0.0,0.0
2,Released,152.0,21.0,1.0,4.0,12.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()